Week 3

<h3>Add News Titles/headlines for the prediction</h3>


<h3>
Using Random Forest to predict if I will lose or gain money
Need the buy price
</h3>
<br>
<h3>
Did not use LSTM
</h3>

In [67]:
#Stock Price Prediction And Forecasting Using Stacked LSTM- Deep Learning

In [68]:
import pandas_datareader as pdr
#using tiingo library
key="08b78fbc98302e995956e2d3b5b8f3c0296d32cc"

I choose these companies because I know them or I read news about them. 
There are no particular reasons

In [119]:

stock = "BURL"#"EW"#"SYK"#"BACHF"#"NTTYY"#"INTC"#"DIS"#"TSLA"#"KEY"#"MSI"#"NVDA"#"AAPL"#"COST"#"AMZN"#"RIVN"#"GLD"#"RPRX"#"WDAY"#"CVX"#"FND"
#"ADBE"#"AAPL"#TDS"#"CLVT"#"RPRX"#"T"#"PHUN"#"ARDX"#"HPQ"#"CLVT"#"LGIH"#"FB"#"ORCL"#"MSFT"#"TDS"#"AMD"#"TM"#"DELL"#"COST"#"ABC"#"ABBV"
df = pdr.get_data_tiingo(stock, api_key=key)

buy = 173.69

In [70]:
df.to_csv(f'{stock}.csv')

In [71]:
import pandas as pd

df = pd.read_csv(f'{stock}.csv')


In [72]:
#Getting the start date of the stock data
import datetime

start_date=df.iloc[0]['date']

#start_date = datetime.datetime(start_date.year, start_date.month, start_date.day)
start_date = start_date.split(' ')
start_date = start_date[0]
start_date #type str

'2016-12-12'

In [73]:
#df.tail()

#Getting the end date of the stock data

end_date=df.iloc[len(df)-1]['date']
#start_date = datetime.datetime(start_date.year, start_date.month, start_date.day)
end_date = end_date.split(' ')
end_date = end_date[0]

end_date #type str

'2021-12-08'

In [75]:
from datetime import datetime

#Convert str format date to datetime object for easy calculation
def str_to_date(strdate):
    return datetime.strptime(strdate, '%Y-%m-%d')

def strD_to_date(strdate):
    return datetime.strptime(strdate, '%m/%d/%Y')

def str_to_str(strdate):
    # 2016-12-09 00:00:00+00:00
    strdate = strdate.split(' ')
    strdate = strdate[0]
    return date_to_str(datetime.strptime(strdate, '%Y-%m-%d'))

#start_date = str_to_date(start_date)


In [76]:
#Convert datetime object to str for easy documentation
def date_to_str(datet):
    return datet.strftime('%m/%d/%Y')

#date_to_str(start_date)

In [77]:
from pynytimes import NYTAPI

#New York Times API
nyt = NYTAPI("9xQE7W9DGO8r6GT8SwCdSTDqEVQTjdO4", parse_dates=True)

In [78]:
#Use pickle library for easy access and store
# 
# Since all news from each month will be the same, so data can be reuse 

import pickle
def save_net(savedfile,path):
    with open(path, 'wb') as f:
        pickle.dump(savedfile, f)
    #print(f'Pickle file saved for Forum {forum_id} at {path}...')
    return path

def get_net(path):
    with open(path, 'rb') as f:
        load = pickle.load(f)
        #print('retrieved!')
    return load


In [93]:
#Return filename in a specific format 
#so I can check if it exist in the file system

def get_file_name(date):
    current = date_to_str(date)
    current = current.split("/")
    current_m, current_yr = current[0],current[2]
    filename = f'{current_m}_{current_yr}_news.pckl'
    #('%m/%d/%Y')
    return filename

def get_yr_from_file(fn):
    current = fn.split("_")
    current_yr = current[1]
    return current_yr

In [94]:
#value to predict
df_closePrice = df.reset_index()[['date','close']]
df_closePrice.head


<bound method NDFrame.head of                            date   close
0     2016-12-12 00:00:00+00:00   87.60
1     2016-12-13 00:00:00+00:00   87.21
2     2016-12-14 00:00:00+00:00   87.23
3     2016-12-15 00:00:00+00:00   87.87
4     2016-12-16 00:00:00+00:00   87.06
...                         ...     ...
1252  2021-12-02 00:00:00+00:00  293.00
1253  2021-12-03 00:00:00+00:00  288.56
1254  2021-12-06 00:00:00+00:00  288.15
1255  2021-12-07 00:00:00+00:00  293.32
1256  2021-12-08 00:00:00+00:00  289.85

[1257 rows x 2 columns]>

<h3>This takes too long to run</h3>

In [118]:
from collections import defaultdict

fn = '01_2017_news.pckl'
def get_news(filename):
    news_result = get_net(filename)
    #Convert the json file into dataframe for easier data process

    headline_df = pd.DataFrame(columns={'date','headlines'})

    data_list = []
    for x in news_result:
        #current_date = date_to_str(x['pub_date']) #str
        current_date = date_to_str(x['pub_date']) #datetime
        day_headline = (x['headline']['main']) 
        if day_headline == '':
            break
        container = {'date':current_date,'headlines':day_headline}
        data_list.append(container)

    headline_df = pd.DataFrame(data_list)

    headlines_dict = defaultdict(list)

    """ for index2, row2 in df_closePrice.iterrows():
       
        if get_yr_from_file(filename) not in str_to_str(row2['date']):
            continue
            
        
        for index, row in headline_df.iterrows():
            if row['date'] == str_to_str(row2['date']):
                headlines_dict[row2['close']].append(row['headlines']) """

    for index, row in headline_df.iterrows():
    
        for index2, row2 in df_closePrice.iterrows():
            
            if get_yr_from_file(filename) not in str_to_str(row2['date']):
                continue
            
            if row['date'] == str_to_str(row2['date']):
                headlines_dict[row2['close']].append(row['headlines'])
                

    print(len(headlines_dict)) #num of days in a month where market open
    
    result = pd.DataFrame(headlines_dict.items(), columns=['close', 'headlines'])
    #result.to_csv("plzzz.csv")
    print(result.head)
    #return headlines_dict

get_news(fn)

20
<bound method NDFrame.head of     close                                          headlines
0   84.78  [Dylann Roof, Charleston Church Killer, Is Dee...
1   86.88  [Call to Centralize Security in Germany Broach...
2   87.13  [Rockefeller Foundation Picks Rajiv J. Shah, a...
3   85.35  [Changes Ahead for Conan O’Brien’s Late-Night ...
4   85.49  [The Rev. Sharonda Coleman-Singleton praying, ...
5   87.39  [Can India Put an End to Identity Politics?, N...
6   85.54  [Trump Received Unsubstantiated Report That Ru...
7   85.31  [Trump Promises Fast Action on Supreme Court N...
8   84.75  [Dance in NYC This Week, Art and Museums in NY...
9   83.87  [Interior Nominee Promotes Navy SEAL Career, W...
10  84.32  [Land Rush in Permian Basin, Where Oil Is Stac...
11  83.33  [For Nicky Jam, a Second Chance at Stardom as ...
12  83.17  [Henry J. Foner, Labor Leader Accused of Commu...
13  82.71  [Protests in Bangladesh Shake a Global Worksho...
14  83.59  [New York Police Plan $275 Million Update

<h3>This will give you extremely LARGE amount of files</h3>
<h4>Data ranging from 2016 to 2021</h4>

In [100]:
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import os.path
from os import path

#NYT API only need month and year
#To ensure the accuracy of the date, I reset the first date to the first day of the month
#Avoiding complcation in the month of Feburary
current_date = str_to_date(start_date).replace(day=1)
a_month = relativedelta(months=1) 
total_dict = {}


#request achieve news only if it does not already exist in the local path
while current_date <=  str_to_date(end_date):
    fn = get_file_name(current_date)
    if not path.isfile(fn):
        newsdata = nyt.archive_metadata(
            current_date
        )
        save_net(newsdata,fn)
    total_dict.update(get_news(fn))
    
    current_date = current_date + a_month

df_data = pd.DataFrame(total_dict.items(), columns=['close', 'headlines'])
df_data.head
print(len(df_data))

28

<h3>
Also practice on the KAggle challenge
</h3>

<h3>
This will only compare/predict to the buy price and its using random forest 
</h3>

In [102]:
from sklearn.model_selection import train_test_split
train = df_data[df_data['date'] < '01/01/2020']
test = df_data[df_data['date'] > '12/31/2019']

train_label = []
test_label = []
for index_temp, row_temp in df_data.iterrows():
    if row_temp['close'] > buy and df_data['date'] < '01/01/2020':
        train_label.append('1')
    elif row_temp['close'] > buy and df_data['date'] > '12/31/2019':
        test_label.append('1')
    elif row_temp['close'] < buy and df_data['date'] > '12/31/2019':
        test_label.append('0')
    else:
        train_label.append('0')


In [ ]:
# Removing special characters
data=train.iloc[:,2:27]
data.replace("[^a-zA-Z]"," ",regex=True, inplace=True)
# Renaming column names for better understanding and ease of access
list1= [i for i in range(25)]
new_Index=[str(i) for i in list1]
data.columns= new_Index
data.head(5)

In [ ]:
for index in new_Index:
    data[index] = data[index].str.lower()
data.head()

In [ ]:
headlines = []
for row in range(0,len(data.index)):
    headlines.append(' '.join(str(x) for x in data.iloc[row,0:25]))

<h3>Processing words from headlines</h3>

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

## implement BAG OF WORDS

countvector=CountVectorizer(ngram_range=(2,2))
traindataset=countvector.fit_transform(headlines)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

seed = 50

## implement RandomForest Classifier
randomclassifier = RandomForestClassifier(
                      min_samples_leaf=50,
                      n_estimators=150,
                      bootstrap=False,
                      oob_score=False,
                      class_weight='balanced_subsample',
                      n_jobs=-1,
                      random_state=seed,
                      max_features='auto')

randomclassifier.fit(traindataset,train['Label'])

In [ ]:
test_transform= []
for row in range(0,len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row,2:27]))
test_dataset = countvector.transform(test_transform)
y_pred = randomclassifier.predict(test_dataset)

In [ ]:
accuracy = accuracy_score(test_label, y_pred)
recall = recall_score(test_label, y_pred)
precision = precision_score(test_label, y_pred)
f1 = f1_score(test_label, y_pred)

print("Random Forest")
print(f"The accuracy of the model is {round(accuracy,3)*100} %")
print(f"The recall of the model is {round(recall,3)*100} %")
print(f"The precision of the model is {round(precision,3)*100} %")
print(f"The f1 of the model is {round(f1,3)}")

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, roc_auc_score, roc_curve, f1_score
import itertools
import matplotlib.pyplot as plt

def evaluate_model(y_pred, probs,train_predictions, train_probs):
    baseline = {}
    baseline['recall']=recall_score(test_label,
                    [1 for _ in range(len(test_label))])
    baseline['precision'] = precision_score(test_label,
                    [1 for _ in range(len(test_label))])
    baseline['roc'] = 0.5
    results = {}
    results['recall'] = recall_score(test_label, y_pred)
    results['precision'] = precision_score(test_label, y_pred)
    results['roc'] = roc_auc_score(test_label, probs)
    train_results = {}
    train_results['recall'] = recall_score(train_label,       train_predictions)
    train_results['precision'] = precision_score(train_label, train_predictions)
    train_results['roc'] = roc_auc_score(train_label, train_probs)
    for metric in ['recall', 'precision', 'roc']: 
          print(f'{metric.capitalize()}\
                 Baseline: {round(baseline[metric], 2)} \
                 Test: {round(results[metric], 2)} \
                 Train: {round(train_results[metric], 2)}')
    # Calculate false positive rates and true positive rates
    base_fpr, base_tpr, _ = roc_curve(test_label, [1 for _ in range(len(test_label))])
    model_fpr, model_tpr, _ = roc_curve(test_label, probs)
    plt.figure(figsize = (8, 6))
    plt.rcParams['font.size'] = 16
    # Plot both curves
    plt.plot(base_fpr, base_tpr, 'b', label = 'baseline')
    plt.plot(model_fpr, model_tpr, 'r', label = 'model')
    plt.legend()
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate'); plt.title('ROC Curves')
    plt.show()

def plot_confusion_matrix(cm, classes, normalize = False,
                          title='Confusion matrix',
                          cmap=plt.cm.Greens): # can change color 
    plt.figure(figsize = (6, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, size = 12)
    plt.colorbar(aspect=5)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, size = 8)
    plt.yticks(tick_marks, classes, size = 8)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    # Label the plot
    for i, j in itertools.product(range(cm.shape[0]),   range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), 
             fontsize = 10,
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")
    plt.grid(None)
    plt.tight_layout()
    plt.ylabel('True label', size = 10)
    plt.xlabel('Predicted label', size = 10)
    plt.show()


#evaluate_model(y_pred,probs,train_predictions,train_probs)

#cm = confusion_matrix(test_label, y_pred)
#plot_confusion_matrix(cm, classes = ['0 - Gain', '1 - Lose'],
#                      title = 'Stock Confusion Matrix')

<h1>Week 4 Conclusion</h1>

<h3>
I was not able to get the shape of the LSTM model to fit

I was able to combine the headlines of the past, but it takes too long to process

I was not able to make the model run on the full data because it needs too much CPU power
</h3>

<h3>
If more time I want to improve my algorithm that search for match date in the headline

Currently, I am using nested for loop that makes the time complexity sad

I think switching to dictionary or use numpy array will give me a much better result
</h3>